In [ ]:
#System installs
!pip install GitPython
!pip install embed
!pip install torch
!pip install transformers==4.46.0
!pip install peft==0.10.0
!pip install --upgrade pymilvus
!pip install numpy==2.0.0


  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.0
    Uninstalling numpy-2.0.0:
      Successfully uninstalled numpy-2.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
Reason for being yanked: This version unfortunately does not work with 3.8 but we did not drop the support yet
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 105.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    

  Using cached numpy-2.0.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
infinity-emb 0.0.58 requires numpy<2,>=1.20.0, but you have numpy 2.0.0 which is incompatible.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read)

In [ ]:
#Imports
from embed import BatchedInference
from concurrent.futures import Future


#Define custom embedding function
register = BatchedInference(
        model_id=[
            "jinaai/jina-embeddings-v2-base-code",
            #"voyageai/voyage-code-3", #not open source--requires separate api key
            "Salesforce/codet5p-110m-embedding",
            "codesage/codesage-large-v2"
        ],
        engine="torch",
        device="cuda",
    )

#Milvus setup
from pymilvus import MilvusClient
milvus_client = MilvusClient(uri="./milvus_demo.db")

#Define collection names and drop if present
collections = ["PriA", "PriB", "PriC", "FrkA", "FrkB", "FrkC"]
for idx in collections:
    if milvus_client.has_collection(idx):
        milvus_client.drop_collection(idx)

#Create new collections
for idx in collections:
    milvus_client.create_collection(
        collection_name=idx,
        metric_type="COSINE",
        consistency_level="Strong",
    )

#Define loading function for repository data
#aiming to successfully store embeddings at first. But want to figure out how to
#store the relevant file contents for each file as metadata on the vector as well, so it can be used in visualization

def load(dir, db, model):
    import os
    for root, dirs, files in os.walk(dir):
        for dir in dirs:
            load(dir, db, model)
        for file in files:
            future: "Future" = register.embed(
                sentences=file, model_id=model
            )
            milvus_client.insert(collection_name=db, data=future.result())

#Pull down repositories
from git import Repo

#embed repositories
primary = Repo.clone_from({"https://github.com/cowsay-org/cowsay"}, "./data/primary")
fork = Repo.clone_from({"https://github.com/cowsay-org/homebrew-cowsay"}, "./data/fork")

load("./data/primary", "PriA", "jina-embeddings-v2-base-code")

register.stop()



INFO     2025-06-09 12:55:02,639 infinity_emb INFO: Started Background Event Loop                 ]8;id=492188;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/sync_engine.py\sync_engine.py]8;;\:]8;id=397844;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/sync_engine.py#57\57]8;;\

INFO     2025-06-09 12:55:02,646 infinity_emb INFO: model=`jinaai/jina-embeddings-v2-base-code`  ]8;id=855015;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/inference/select_model.py\select_model.py]8;;\:]8;id=431852;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/inference/select_model.py#66\66]8;;\
         selected, using engine=`torch` and device=`cuda`                                                          

INFO     2025-06-09 12:55:02,706 sentence_transformers.SentenceTransformer INFO: Load    ]8;id=213408;file:///usr/local/lib/python3.11/dist-packages/sentence_transformers/SentenceTransformer.py\SentenceTransformer.py]8;;\:]8;id=263057;file:///usr/local/lib/python3.11/dist-packages/sentence_transformers/SentenceTransformer.py#218\218]8;;\
         pretrained SentenceTransformer: jinaai/jina-embeddings-v2-base-code                                       

INFO     2025-06-09 12:55:04,122 infinity_emb INFO: Closed Background Event Loop                  ]8;id=374771;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/sync_engine.py\sync_engine.py]8;;\:]8;id=240528;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/sync_engine.py#65\65]8;;\

INFO     2025-06-09 12:55:04,222 infinity_emb INFO: Closed Background Event Loop                  ]8;id=459279;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/sync_engine.py\sync_engine.py]8;;\:]8;id=44308;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/sync_engine.py#65\65]8;;\

...

The class `optimum.bettertransformers.transformation.BetterTransformer` is deprecated and will be removed in optimum v2.0.


ERROR    2025-06-09 12:55:06,914 infinity_emb ERROR: BetterTransformer is not available for      ]8;id=189859;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/transformer/acceleration.py\acceleration.py]8;;\:]8;id=377150;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/transformer/acceleration.py#64\64]8;;\
         model: <class                                                                                             
         'transformers_modules.jinaai.jina-bert-v2-qk-post-norm.3baf9e3ac750e76e8edd301917017688                   
         4695fb94.modeling_bert.JinaBertModel'> The transformation of the model JinaBertModel to                   
         BetterTransformer failed while it should not. Please fill a bug report or open a PR to                    
         support this model at https://github.com/huggingface/optimum/. Continue without                           
         bettertransformer modeling code.                                                                          
         Traceback (most recent call last):                                                                        
           File                                                                                                    
         "/usr/local/lib/python3.11/dist-packages/infinity_emb/transformer/acceleration.py",                       
         line 60, in to_bettertransformer                                                                          
             model = BetterTransformer.transform(model)                                                            
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                                            
           File "/usr/lib/python3.11/contextlib.py", line 81, in inner                                             
             return func(*args, **kwds)                                                                            
                    ^^^^^^^^^^^^^^^^^^^                                                                            
           File                                                                                                    
         "/usr/local/lib/python3.11/dist-packages/optimum/bettertransformer/transformation.py",                    
         line 292, in transform                                                                                    
             set_last_layer(model_fast)                                                                            
           File                                                                                                    
         "/usr/local/lib/python3.11/dist-packages/optimum/bettertransformer/transformation.py",                    
         line 172, in set_last_layer                                                                               
             raise Exception(                                                                                      
         Exception: The transformation of the model JinaBertModel to BetterTransformer failed                      
         while it should not. Please fill a bug report or open a PR to support this model at                       
         https://github.com/huggingface/optimum/                                                                   

INFO     2025-06-09 12:55:06,920 infinity_emb INFO: Switching to half() precision (cuda: ]8;id=6461;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/transformer/embedder/sentence_transformer.py\sentence_transformer.py]8;;\:]8;id=631529;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/transformer/embedder/sentence_transformer.py#84\84]8;;\
         fp16).                                                                                                    

INFO     2025-06-09 12:55:06,934 infinity_emb INFO: model=`voyageai/voyage-code-3` selected,     ]8;id=469922;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/inference/select_model.py\select_model.py]8;;\:]8;id=818722;file:///usr/local/lib/python3.11/dist-packages/infinity_emb/inference/select_model.py#66\66]8;;\
         using engine=`torch` and device=`cuda`                                                                    

EntryNotFoundError: 404 Client Error. (Request ID: Root=1-6846d9aa-59b1dc3646778c3159cda1a5;056d5d2d-626e-4bf8-8fb0-e2396abe79db)

Entry Not Found for url: https://huggingface.co/voyageai/voyage-code-3/resolve/main/config.json.

In [ ]:
!rm -rf ./repo